In [1]:
# -*- coding: utf-8 -*-
import os
import requests
import xlrd,xlwt
from xlutils.copy import copy

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# 设置常量
PATH = os.path.abspath(r"D:\玖隆\物贸公司\报表\临时报表")
FILENAME = "螺纹钢盘螺价格.xlsx"
SHEETNAME = "url_map"
OUTCOME_FILE = "螺纹钢盘螺价格_爬虫抓取数据.xlsx"

BASE_FILE = os.path.join(PATH,FILENAME)
print(BASE_FILE)

NEED_NUM = 90  # 需要的记录数

CHROME_DRIVER = 'D:\Python\驱动\chromedriver_win32\chromedriver.exe'


D:\玖隆\物贸公司\报表\临时报表\螺纹钢盘螺价格.xlsx


In [3]:
nrows = 0 
ncols = 0 
data = None

In [4]:
def open_para_xlsx(path=PATH, filename=FILENAME, sheetname=SHEETNAME):
    # 打开Excel对应的参数需求表
    data = xlrd.open_workbook(os.path.join(path,filename))
    table = data.sheet_by_name(sheetname)  # 返回一个xlrd.sheet.Sheet()对象
    data.sheet_loaded(sheetname)  # 检查某个sheet是否导入完毕
    
    nrows = table.nrows  #获取该sheet中的有效行数
    ncols = table.ncols  # 有效列数
    return table

In [5]:
table = open_para_xlsx()

In [6]:
# 将参数数据加载进内存
para_dic = {}

for i in range(NEED_NUM):
    para_dic[int(table.cell_value(i+2,6))] = {"品名":table.cell_value(i+2,7),
                                         "二级入口网址":table.cell_value(i+2,2),
                                         "二级入口网址查找对应表名":table.cell_value(i+2,5),
                                         "型号":table.cell_value(i+2,8),
                                         "规格":table.cell_value(i+2,9),
                                         "钢厂":table.cell_value(i+2,10),
                                         "CTR值":table.cell_value(i+2,11),
                                         "匹配原始数据值":None,
                                         "是否找到对应表名":False,
                                         "更新成功":False
                                        }

In [7]:
def get_specfic_table_link(table_content,match_title):
    match_a = table_content.findAll("a",title=match_title)  # 在获取到的页面的bs4对象上，继续找到对应a标签对象
    
    next_a_href_suffix = table_content.findAll("a",title=match_title)[0].get("href")
    if next_a_href_suffix.startswith("http"):
        next_a_href = next_a_href_suffix
    else:
        next_a_href = "https:" + next_a_href_suffix
    return next_a_href

In [8]:
def find_content_href(index,para_dic=para_dic):
    url = para_dic.get(index).get("二级入口网址")
    match_title = para_dic.get(index).get("二级入口网址查找对应表名")
    response = requests.get(url)
    html_doc = response.text 
    soup = BeautifulSoup(html_doc,"html.parser")
    table_content = soup.find("ul",class_="nlist")
    
    # 判断是否在第一页就能找到对应的 表名 a标签
    titles = table_content.get_text().replace("\t","").replace("\n","")
    
    if titles.find(match_title) == -1:
        # 找第二页
        find_index = soup.find("div",class_="page")
        page2 = find_index.find("a")
        if page2.contents[0] == "2":
            page2_change = find_index.find("a")["href"].split("-")[-1]
            page2_url = url.replace("1.html",page2_change)
            response2 = requests.get(page2_url)
            soup2 = BeautifulSoup(response2.text,"html.parser")
            table_content2 = soup2.find("ul",class_="nlist")
            titles2 = table_content2.get_text().replace("\t","").replace("\n","")
            if titles2.find(match_title) == -1:
                print("今日数据尚未更新")
                return para_dic
            else: 
                para_dic[index]["是否找到对应表名"] = True
                para_dic[index]["每日新增数据的网址"] = get_specfic_table_link(table_content2,match_title)
    else:
        # 如果找到了指定的表名
        # 则解析获取对应的a标签的href网址
        # 返回对应的表名的 href 字符串
        para_dic[index]["是否找到对应表名"] = True
        para_dic[index]["每日新增数据的网址"] = get_specfic_table_link(table_content,match_title) 
        return para_dic

In [9]:
# 获取对应的新增网址，所有网址信息都输出，则说明今天数据都更新了
for i in range(1,91):
    try:
        if para_dic.get(i).get("CTR值") is not None:
            find_content_href(i) 
    except Exception as e:
        print(e)

In [10]:
for k,v in para_dic.items():
    print(k,v["是否找到对应表名"],v.get("每日新增数据的网址"))

1 True https://jiancai.mysteel.com/m/20/0715/09/5E698001CBDA11F5.html
2 True https://jiancai.mysteel.com/m/20/0715/09/5E698001CBDA11F5.html
3 True https://jiancai.mysteel.com/m/20/0715/09/5E698001CBDA11F5.html
4 True https://jiancai.mysteel.com/m/20/0715/12/2E3ADD842ED4852F.html
5 True https://jiancai.mysteel.com/m/20/0715/12/2E3ADD842ED4852F.html
6 True https://jiancai.mysteel.com/m/20/0715/12/2E3ADD842ED4852F.html
7 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
8 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
9 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
10 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
11 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
12 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
13 True https://jiancai.mysteel.com/m/20/0715/09/6DAE9CB4E2F9983F.html
14 True https://jiancai.mysteel.com/m/20/0715/09/6DAE9CB4E2F9983F.html
15 True https:/

In [11]:
for k,v in para_dic.items():
    
    localtime = time.localtime(time.time())
    if v.get("每日新增数据的网址") is not None:
        extract_month = int(v.get("每日新增数据的网址").split("/")[5][:2])
        
        if extract_month == localtime.tm_mon:
            print(k,v.get("是否找到对应表名"),v.get("每日新增数据的网址"))
        else: 
            para_dic.get(k)["是否找到对应表名"] = False
            para_dic.get(k)["每日新增数据的网址"] = None
            print(k,v.get("是否找到对应表名"),v.get("每日新增数据的网址"),"不是当月网址")
    else:
        print(k,v.get("是否找到对应表名"),v.get("每日新增数据的网址"))
        

1 True https://jiancai.mysteel.com/m/20/0715/09/5E698001CBDA11F5.html
2 True https://jiancai.mysteel.com/m/20/0715/09/5E698001CBDA11F5.html
3 True https://jiancai.mysteel.com/m/20/0715/09/5E698001CBDA11F5.html
4 True https://jiancai.mysteel.com/m/20/0715/12/2E3ADD842ED4852F.html
5 True https://jiancai.mysteel.com/m/20/0715/12/2E3ADD842ED4852F.html
6 True https://jiancai.mysteel.com/m/20/0715/12/2E3ADD842ED4852F.html
7 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
8 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
9 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
10 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
11 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
12 True https://jiancai.mysteel.com/m/20/0715/09/47103072A261351D.html
13 True https://jiancai.mysteel.com/m/20/0715/09/6DAE9CB4E2F9983F.html
14 True https://jiancai.mysteel.com/m/20/0715/09/6DAE9CB4E2F9983F.html
15 True https:/

# 最关键的价格数据是ajax返回的，通过selenium模拟服务器，获取ajax数据

In [12]:
#指定chrome的驱动
#执行到这里的时候Selenium会到指定的路径将chrome driver程序运行起来
driver = webdriver.Chrome(CHROME_DRIVER)
if para_dic.get(1).get("每日新增数据的网址") is not None:
    driver.get(para_dic.get(1).get("每日新增数据的网址"))
else:
    driver.get("https://jiancai.mysteel.com/m/20/0527/09/9C38A68E4E18B9D3.html")
    
time.sleep(10)
for k,v in para_dic.items():
    #get 方法 打开指定网址
    if para_dic.get(k).get("CTR值") != '':
        driver.get(para_dic.get(k).get("每日新增数据的网址"))
        id_ctr_value = para_dic.get(k).get("CTR值")
        #选择网页元素
        element_keyword = driver.find_element_by_id(id_ctr_value)
        para_dic[k]["匹配原始数据值"]=element_keyword.text
    time.sleep(1)

In [13]:
for k,v in para_dic.items():
    print(k,v.get("匹配原始数据值"))

1 螺纹钢 Ф16-25 HRB400 沙钢 3710 - Ф16:3740;Ф25:3760 代理
2 螺纹钢 Ф16-25 HRB400 永钢 3690 - Ф16:3720;Ф25:3740 代理
3 螺纹钢 Ф16-25 HRB400 房铁市公
中天 3690 - Ф16:3720;Ф25:3740 代理
4 盘螺 Ф8-10 HRB400 沙钢 3950 -10 Ф10:4000 现货
5 盘螺 Ф8-10 HRB400 永钢 3950 -10 Ф10:4000 现货
6 盘螺 Ф8-10 HRB400 房铁市公
中天 3950 -10 Ф10:4000 代理
7 螺纹钢 Ф16-25 HRB400 沙钢 3740 - Ф16:3770;Ф25:3770 现货
8 螺纹钢 Ф16-25 HRB400 永钢 3730 - Ф16:3760;Ф25:3760 现货
9 螺纹钢 Ф16-25 HRB400 中天 3730 - Ф16:3760;Ф25:3760 现货
10 盘螺 Ф8-10 HRB400 沙钢 3990 - Ф8:4020 现货
11 盘螺 Ф8-10 HRB400 永钢 3980 - Ф8:4010 现货
12 盘螺 Ф8-10 HRB400 中天 3980 - Ф8:4010 现货
13 螺纹钢 Ф16-25 HRB400 沙钢 3840 - Ф16:3860;Ф25:3860 现货
14 螺纹钢 Ф16-25 HRB400 永钢 3840 - Ф16:3860;Ф25:3860 代理
15 螺纹钢 Ф16-25 HRB400 中天 3830 - Ф16:3850;Ф25:3850 代理
16 盘螺 Ф8-10 HRB400 沙钢 4050 - 现货
17 盘螺 Ф8-10 HRB400 永钢 4050 - 代理
18 盘螺 Ф8-10 HRB400 中天 4030 - 现货
19 螺纹钢 Ф16-25 HRB400 沙钢 3780 - Ф22:3810;Ф25:3810 代理
20 螺纹钢 Ф16-25 HRB400 永钢 3770 - Ф22:3800;Ф25:3800 现货
21 螺纹钢 Ф16-25 HRB400 中天 3770 - Ф22:3800;Ф25:3800 代理
22 盘螺 Ф8-10 HRB400 沙钢 4020 - 代

In [14]:
# 打开工作表，保持原表的格式
rb = xlrd.open_workbook(os.path.join(PATH,OUTCOME_FILE))
wb = copy(rb)
ws = wb.get_sheet("sheet1")

for i in range(1,91):
    if para_dic.get(i).get("是否找到对应表名"):
        ws.write(i-1,0,i)
        ws.write(i-1,1,para_dic.get(i).get("是否找到对应表名"))
        ws.write(i-1,2,para_dic.get(i).get("匹配原始数据值"))
    
wb.save(os.path.join(PATH,OUTCOME_FILE)) 